In [1]:
import csv
import itertools
from collections import defaultdict
from typing import DefaultDict, Dict, List, Set, Tuple

In [2]:
data: DefaultDict[str, DefaultDict[float, DefaultDict[int, DefaultDict[int, List[Tuple[float, float]]]]]] = defaultdict(
    lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
)
possible: Tuple[Set[str], Set[float], Set[int], Set[int]] = (set(), set(), set(), set())
with open("vrpdfd-summary.csv", "r", encoding="utf-8", newline="") as csvfile:
    csvreader = csv.reader(csvfile, delimiter=",", quotechar="\"")
    csvlines = iter(csvreader)
    next(csvlines)  # Skip header

    for row in csvlines:
        problem = row[0]
        mutation_rate = float(row[3])
        reset_after = int(row[4])
        local_search_batch = int(row[6])
        profit = float(row[7])
        computation_time = float(row[11])

        possible[0].add(problem)
        possible[1].add(mutation_rate)
        possible[2].add(reset_after)
        possible[3].add(local_search_batch)
        data[problem][mutation_rate][reset_after][local_search_batch].append((profit, computation_time))

possible

({'50.10.1',
  '50.10.3',
  '50.20.1',
  '50.20.3',
  '50.30.1',
  '50.30.3',
  '50.40.1',
  '50.40.3'},
 {0.1, 0.5, 0.9},
 {10, 15, 20},
 {20, 50, 80})

In [3]:
sums: DefaultDict[str, List[float]] = defaultdict(list)
for problem, mutation_rate, reset_after, local_search_batch in itertools.product(*possible):
    sums[problem].extend(profit for profit, _ in data[problem][mutation_rate][reset_after][local_search_batch])

average: Dict[str, float] = {}
for problem, s in sums.items():
    average[problem] = sum(s) / len(s)

average

{'50.30.1': -41288.32555555556,
 '50.30.3': -33825.53277777779,
 '50.20.3': 1640.9681481481523,
 '50.40.1': -104734.75259259259,
 '50.20.1': 1714.720555555556,
 '50.40.3': -75660.03851851846,
 '50.10.1': 25017.75314814814,
 '50.10.3': 27184.65129629628}

In [4]:
normalized_data: DefaultDict[str, DefaultDict[float, DefaultDict[int, Dict[int, float]]]] = defaultdict(
    lambda: defaultdict(lambda: defaultdict(dict))
)
max_data: DefaultDict[str, DefaultDict[float, DefaultDict[int, Dict[int, float]]]] = defaultdict(
    lambda: defaultdict(lambda: defaultdict(dict))
)

for problem, mutation_rate, reset_after, local_search_batch in itertools.product(*possible):
    profits = [profit for profit, _ in data[problem][mutation_rate][reset_after][local_search_batch]]
    normalized_data[problem][mutation_rate][reset_after][local_search_batch] = (sum(profits) / len(profits) - average[problem]) / abs(average[problem])
    max_data[problem][mutation_rate][reset_after][local_search_batch] = (max(profits) - average[problem]) / abs(average[problem])

In [5]:
normalized_improved: Dict[Tuple[float, int, int], float] = {}
for mutation_rate, reset_after, local_search_batch in itertools.product(*possible[1:]):
    values: List[float] = [normalized_data[problem][mutation_rate][reset_after][local_search_batch] for problem in possible[0]]
    normalized_improved[mutation_rate, reset_after, local_search_batch] = sum(values) / len(values)

normalized_improved

{(0.1, 10, 80): 0.813408666481487,
 (0.1, 10, 50): 0.6827108838399916,
 (0.1, 10, 20): 0.5421534945217578,
 (0.1, 20, 80): -0.6092462800257489,
 (0.1, 20, 50): -0.7468513303801315,
 (0.1, 20, 20): -0.7071080484322034,
 (0.1, 15, 80): -0.1642705703782099,
 (0.1, 15, 50): -0.0931359757090737,
 (0.1, 15, 20): 0.024959649349843217,
 (0.5, 10, 80): 0.42270460092792644,
 (0.5, 10, 50): 0.5584715913603281,
 (0.5, 10, 20): 0.6273103924215566,
 (0.5, 20, 80): -0.5882734968309051,
 (0.5, 20, 50): -0.5807847435463565,
 (0.5, 20, 20): -0.3348013105984479,
 (0.5, 15, 80): -0.19263411642195732,
 (0.5, 15, 50): -0.06259301431084931,
 (0.5, 15, 20): -0.25365257476289027,
 (0.9, 10, 80): 0.7237488473995,
 (0.9, 10, 50): 0.7382224915021389,
 (0.9, 10, 20): 0.45335758690712774,
 (0.9, 20, 80): -0.40189931829454933,
 (0.9, 20, 50): -0.4604150639573386,
 (0.9, 20, 20): -0.40248616715509566,
 (0.9, 15, 80): 0.04691770521351675,
 (0.9, 15, 50): -0.011206161442589372,
 (0.9, 15, 20): -0.02460773767882326}

In [6]:
max_improved: Dict[Tuple[float, int, int], float] = {}
for mutation_rate, reset_after, local_search_batch in itertools.product(*possible[1:]):
    values: List[float] = [max_data[problem][mutation_rate][reset_after][local_search_batch] for problem in possible[0]]
    max_improved[mutation_rate, reset_after, local_search_batch] = sum(values) / len(values)

max_improved

{(0.1, 10, 80): 1.4941808909128995,
 (0.1, 10, 50): 1.4006165571720306,
 (0.1, 10, 20): 1.3680793481723068,
 (0.1, 20, 80): 0.5907479589101973,
 (0.1, 20, 50): 0.27639485474304326,
 (0.1, 20, 20): 0.4276068281677539,
 (0.1, 15, 80): 1.0885251726223315,
 (0.1, 15, 50): 0.9907641564867715,
 (0.1, 15, 20): 1.0348402757813944,
 (0.5, 10, 80): 1.3106409323372996,
 (0.5, 10, 50): 1.1881921113429321,
 (0.5, 10, 20): 1.4271964395622712,
 (0.5, 20, 80): 0.6421297398507653,
 (0.5, 20, 50): 0.6746216744874999,
 (0.5, 20, 20): 0.9412482984607421,
 (0.5, 15, 80): 0.8597193939907609,
 (0.5, 15, 50): 1.0901252200990892,
 (0.5, 15, 20): 1.0027922345731328,
 (0.9, 10, 80): 1.4686635171904376,
 (0.9, 10, 50): 1.5186452974259095,
 (0.9, 10, 20): 1.454935048038959,
 (0.9, 20, 80): 0.8377186888306608,
 (0.9, 20, 50): 0.8837429419585765,
 (0.9, 20, 20): 0.979536488177732,
 (0.9, 15, 80): 0.9796019819212152,
 (0.9, 15, 50): 1.0829304234266541,
 (0.9, 15, 20): 0.758204686065323}

In [7]:
max(itertools.product(*possible[1:]), key=normalized_improved.__getitem__)

(0.1, 10, 80)

In [8]:
max(itertools.product(*possible[1:]), key=max_improved.__getitem__)

(0.9, 10, 50)